Class Assignment 1: CNN on CIFAR-10 data.

Total Marks: 30


Import the necessary python packages and libraries in the following cell.

In [2]:
## Import the libraries here in this cell
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt




In [3]:

resnet18 = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 149MB/s]


Q1 (a) Print the torch and torchvision version [Marks: 1+1]

In [4]:
# Your code here
print(torch.version )
print(torchvision.version )

<module 'torch.version' from '/usr/local/lib/python3.10/dist-packages/torch/version.py'>
<module 'torchvision.version' from '/usr/local/lib/python3.10/dist-packages/torchvision/version.py'>


Q1 (b) Set the device to cuda, if available. [Marks: 1]

In [5]:
# device = // Your code here
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Q1 (c) Transform or preprocess the data utilizing the following operations. [Marks: 1x5 = 5]

*   Normalize data
*   Resize each image to 224 x 224
*   Add Color Jitter with hue and saturation as 0.5
*   Introduce Random horizontal flips
*   Rotate images about 20 degrees


In [6]:
# transform = // Your code here'
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(hue=0.5, saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Q2 (a) Train and test Resnet18 on Even numbered Classes in CIFAR10 Dataset. [Marks: 5]

In [7]:
# Filter only even-numbered classes

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)

# Your code here
#HINT: Use torch utils Subset class to create a subset of the dataset

100%|██████████| 170498071/170498071 [00:02<00:00, 64827849.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
# even_classes = []
# for inputs, labels in trainloader:
#   if labels%2 == 0:
#     even_classes.append(labels)


In [9]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)



In [10]:
model = models.resnet18(weights='DEFAULT').to(device)
model.fc = nn.Linear(model.fc.in_features, 10).to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Q2 (b) Complete the code for backpropagation inside the train function. [Marks: 5]


In [12]:
def train(model, trainloader, criterion, optimizer, device):

    model.train()

    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in trainloader:

        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, pred = outputs.max(1)
        total_train += labels.size(0)
        correct_train += pred.eq(labels).sum().item()

    train_acc = 100 * correct_train / total_train

    return train_loss/len(trainloader), train_acc

Q2 (c) Complete the test function to test out the model. [Marks: 5]

In [13]:
def test(model, testloader, criterion, device):
    model.eval()
    correct_test = 0
    total_test = 0

    # Your code here
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    test_accuracy = 100 * correct_test / total_test
    return test_accuracy

Q3 (a) Complete the code to plot the train and test accuracies [Marks: 2]

In [14]:
def plot_accuracies(train_accuracies, test_accuracies, epochs):

    # Your code here
    plt.plot(range(1, epochs + 1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, epochs + 1), test_accuracies, label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Train and Test Accuracies vs Epochs')
    plt.legend()
    plt.show()


Q3 (b) Complete the code to plot a Confusion Matrix (without sklearn) [Marks: 3]

In [15]:
import seaborn as sns

def plot_confusion_matrix(model, dataloader, class_names, device):
    model.eval()
    all_labels = []
    all_predictions = []

    num_classes = len(class_names)
    confusion_matrix = torch.zeros(num_classes, num_classes)
    confusion_matrix_result = confusion_matrix(all_labels, all_predictions)

    # Your code here
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
    plt.figure(figsize=(num_classes, num_classes))
    sns.heatmap(confusion_matrix, annot=True, fmt=".0f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


Q3 (c) Complete the following code block to train and test the model. [Marks: 2]

In [ ]:
epochs = 10
train_accuracies = []
test_accuracies = []

for epoch in range(epochs):

        # Complete the loop
        train_loss, train_acc = train(model, trainloader, criterion, optimizer, device)
        train_accuracies.append(train_acc)


        test_acc = test(model, testloader, criterion, device)
        test_accuracies.append(test_acc)

        print(f'Epoch {epoch + 1} | Train Loss: {train_loss:.3f} | Train Accuracy: {train_acc:.3f}% | Test Accuracy: {test_acc:.3f}%')


plot_accuracies(train_accuracies, test_accuracies, epochs)
class_names = ['0', '2', '4', '6', '8']
plot_confusion_matrix(model, testloader, class_names, device)

Epoch 1 | Train Loss: 0.000 | Train Accuracy: 74.308% | Test Accuracy: 78.820%
